In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from IPython.display import clear_output
import seaborn as sns

import cPickle as pk
import pandas as pd
import networkx as nx
import scipy.sparse as sp
import itertools as itls
from itertools import combinations
import numpy as np
from math import sqrt

import sys

## Importing
[to top](#Table-of-Contents) 

In [ ]:
pwd

In [ ]:
df = pk.load(open('pickles_and_munged_csv/pre1950_cated_cumed_codes_as_ints_v1.pk', 'rb')); df.head()

In [ ]:
df.drop('SU', axis=1, inplace=True)

In [ ]:
print df.columns.values

# AdjM building

# DeltaCon

In [ ]:
def compute_FBP_invs(A):

    # max power to calculate inverse to
    max_pw = 7

    # make identity
    I = sp.identity(A.shape[0])
 
   # remove self links - can't propogate beliefs to self - need to consider if the self links should be counted in
    # degree still?
    A = A - sp.dia_matrix((A.diagonal(), [0]),
                                    shape=(A.shape[0], A.shape[0]))

    # make degree matrix
    ds = [int(x) for x in A.sum(axis=1)]
    D = sp.dia_matrix((ds, [0]),
                     shape=(len(ds), len(ds)))
    
    # compute about-half homophility factor that gaurentees convergences of W power series
    c1 = D.sum() + 2
    c2 = np.dot(D,D).sum() - 1
    h_h = sqrt((-c1+sqrt(c1**2 + 4*c2))
               /(8*c2)
              )
    
    # compute system constants ah (from ahD) and ch from (achA)
    ah = 4*(h_h**2) /(1-4*(h_h**2))
    ch = 2*h_h / (1-4*(h_h**2))
    
    # create matrix M to form (I - M)^-1
    M = ch*A - ah*D
    
    # expand to power series to compute inverse
    inv = I
    mat = M
    pwr = 1
    while mat.data[mat.data.argmax()] > 10**(-9) and pwr < max_pw:
        inv = inv + mat
        mat = mat * M
        pwr = pwr + 1
    return inv

In [ ]:
def DeltaCon(A1, A2):
    
    # initialize
    # prior belief for nodes (doesn't really affect results of deltacon)
    p = 0.501
    
    # Solve the linear BP systems (FaBP): inv corresponds to [I + ah*D - ch*A]^-1
    # and multiple by about-half prior beliefs
    inv1 = compute_FBP_invs(A1) * (p-0.5)
    inv2 = compute_FBP_invs(A2) * (p-0.5)
    
    # compute the similarities using Matusita distance
    diffM = (inv1.sqrt() - inv2.sqrt()) ** 2
    # delete diagonal entries -  the influence nodes have on themselves
    diffM = diffM - sp.dia_matrix((diffM.diagonal(), [0]), shape=(diffM.shape[0], diffM.shape[0]))
    return 1 / (1 + sqrt(diffM.sum()))

# Applying

In [ ]:
def compute_DeltaCon_sims(df, year):
    df_temp = pd.DataFrame()
    for col1 in df:
        print '\n%s' % col1
        if len(df.loc[year,col1]) == 0:
            for col2 in df:
                df_temp.loc[col1,col2] = -1
        else:
            g1 = nx.MultiGraph()
            for ls in df.loc[year,col1]:
                g1.add_edges_from(combinations(ls, 2))

            for col2 in df:
                print col2,
                if len(df.loc[year,col2]) == 0:
                    df_temp.loc[col1,col2] = -1
                else:
                    g2 = nx.MultiGraph()
                    for ls in df.loc[year,col2]:
                        g2.add_edges_from(combinations(ls, 2))
                    
                    
                        
                    df_temp.loc[col1,col2] = DeltaCon(nx.adj_matrix(g1),nx.adj_matrix(g2))
    return df_temp

In [ ]:
DelCon_test_v1 = compute_DeltaCon_sims(df,1960)

# Timing

In [ ]:
%%timeit
g_US = nx.MultiGraph()
for ls in df.loc[2010,'US']:
    g_US.add_edges_from(combinations(ls, 2))

In [ ]:
%%timeit
for ls in df.loc[2010,'US']:
    combinations(ls, 2)

In [ ]:
g_US = nx.MultiGraph()
for ls in df.loc[2010,'US']:
    g_US.add_edges_from(combinations(ls, 2))

In [ ]:
%%timeit
A_US = nx.adj_matrix(g_US)

In [ ]:
%time A_US = nx.adj_matrix(g_US)

In [ ]:
print sys.getsizeof(A_US)
print sys.getsizeof(g_US)
print sys.getsizeof()

In [ ]:
g_US

In [ ]:
A_US

In [ ]:
print A_US
print A_US[0,23]

In [ ]:
A_US[:,:]

In [ ]:
who